In [1]:
""" Import dependencies """
import os
import keras
import numpy as np
import tensorflow as tf
from sklearn import metrics
from ML.ml_utils import ML_utils
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline

Segmentation Models: using `keras` framework.


In [2]:
""" User defined parameters """
matplotlib.use('agg')
plt.ioff()
dataset_name = 'data_medium'
path_to_model = '/localhome/studenter/mikaellv/Project/ML/models/test_model_5'
save_path_image_predictions = '/localhome/studenter/mikaellv/Project/ML/predicted_images/'

In [3]:
""" Load model """
ml = ML_utils()
model = keras.models.load_model(path_to_model)

""" Define test image paths """
path_to_images = "data/datasets/" + dataset_name + "/test/images"
path_to_masks = "data/datasets/" + dataset_name + "/test/masks"

""" Define classes """
classes = {
    0: 'not_water',
    1: 'water'
}

2022-03-27 12:17:49.383300: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 12:17:49.411756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 12:17:49.412493: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 12:17:49.413518: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
""" Produce images """
n_images = len([img for img in os.listdir(path_to_images) if not img.startswith('.')])
test_data_generator = ml.DataGenerator(path_to_images,path_to_masks,train=False)
n_classes = len(classes)
predict_batch = 32
n_batch = n_images // 32
if not n_images % 32 == 0: n_batch += 1
images = []
predictions = []
segmentations = []

""" Predict images """
for batch in range(0,n_batch):
    imgs, masks = next(test_data_generator)
    for i in range(0, len(imgs)):
        img = np.expand_dims(imgs[i],axis=0)
        images.append(imgs[i])
        predictions.append(model.predict(img))
        segmentations.append(masks[i])

2022-03-27 12:19:09.975948: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301
2022-03-27 12:19:10.785628: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-03-27 12:19:10.805509: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-03-27 12:19:10.805541: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-03-27 12:19:10.805642: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

In [5]:
""" Set predictions to maximum value (single-band) """
for i in range(len(predictions)):
    predictions[i] = np.argmax(predictions[i],axis=-1)
    segmentations[i] = np.argmax(segmentations[i],axis=-1)

In [6]:
""" Save predicted images for visualization """
count = 0
for img, pred, mask in zip(images,predictions,segmentations):
    pred = np.rollaxis(pred,0,3)
    fig, axs = plt.subplots(1,3,figsize=(25,25))
    plt.tight_layout()

    axs[0].imshow(img[:,:,0],cmap='PuBuGn_r')
    axs[1].imshow(pred)
    axs[2].imshow(mask)

    axs[0].set_title("Original Image")
    axs[1].set_title("Prediction")
    axs[2].set_title("Ground Truth")

    plt.savefig(save_path_image_predictions + str(count) + '.png')
    count += 1

/tmp/ipykernel_2360079/2660346826.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(1,3,figsize=(25,25))


In [8]:
""" Calculate performance """
# Flatten arrays
print("[INFO] Flattening all numpy arrays... this may take a couple om minutes.")
predictions = np.concatenate([np_array.ravel() for np_array in predictions])
segmentations = np.concatenate([np_array.ravel() for np_array in segmentations])

print("[INFO] Calculating precision/recall...")
precision = metrics.precision_score(segmentations, predictions, average='weighted')
recall = metrics.recall_score(segmentations, predictions, average='weighted')

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {(2*precision*recall)/(recall+precision)}")

Precision: 0.9721709699850013
Recall: 0.9670655425323065
F1-Score: 0.9696115357299029
